<a href="https://colab.research.google.com/github/ykitaguchi77/Strabismus_AI_project/blob/main/ExtractList_from_imgNum1.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import glob
import os
import pandas as pd
import sys
import shutil

#google driveをcolabolatoryにマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pd.set_option('display.max_columns', 10)

def openCSV(csv_path):
    #df = pd.read_csv(csv_path, encoding="shift-jis")
    df = pd.read_csv(csv_path, encoding="utf-8",header=None)
    #df = pd.read_csv(csv_path, encoding="cp932")
    #df = df.drop(df.columns[[0]], axis=1) #1列目を削除する
    #print(df)
    return df

#CSVを開く （csv1:編集するファイル、csv2:参照するファイル）
csv2_path = '/content/drive/MyDrive/Deep_learning/Facephoto_LIST/DiseaseInfo_all_connected_new3.csv'
df2 = openCSV(csv2_path) #CSVを開く
df2

In [3]:
#ファイル名のアルファベットを削除するリネーム
#path = "/content/drive/MyDrive/Deep_learning/Strabismus/dataset(3class_notcut)/train/exte/e126.jpg"
#path = "/content/drive/MyDrive/Deep_learning/Strabismus/dataset(3class_notcut)/train/exte/126.jpg"

img_path = "/content/drive/MyDrive/Deep_learning/Strabismus/eso_250px"

orig_names = glob.glob(img_path + "/*")
for orig_name in orig_names:
    dir_name = os.path.dirname(orig_name)
    base_name = os.path.basename(orig_name)
    
    print(base_name)
    if base_name[0].isalpha() is True:
        base_name = base_name[1:] #アルファベットから始まっているものはアルファベットを削除する
    else:
        base_name = base_name

    dst_name = os.path.join(dir_name, base_name)
    os.rename(orig_name, dst_name)

In [4]:
#パスから画像名のリストを作成
img_path = "/content/drive/MyDrive/Deep_learning/Strabismus/Dataset_250px_20211116"

contlist, extelist, intelist = [], [], []

#path = "/content/drive/MyDrive/Deep_learning/Strabismus/dataset(3class_notcut)/train/exte/e126.jpg"
#このうち126の部分のみを取り出す

cont_name = glob.glob(img_path +"/cont/*")
for i in cont_name:
    contlist.append(int(os.path.basename(os.path.splitext(i)[0])))
contlist=sorted(contlist)

exte_name = glob.glob(img_path +"/exo/*")
for i in exte_name:
    #extelist.append(int(os.path.basename(os.path.splitext(i)[0]).lstrip("e").split("-")[0]))
    extelist.append(int(os.path.basename(os.path.splitext(i)[0])))
extelist=sorted(extelist)
    
inte_name = glob.glob(img_path +"/eso/*")
for i in inte_name:
    intelist.append(int(os.path.basename(os.path.splitext(i)[0])))
intelist=sorted(intelist)

In [5]:
def extract(imgname_list, df_orig):
    #もとのcsvと列名を揃えたデータフレームを作成
    cols =  df_orig.columns
    df_dst = pd.DataFrame(index=[], columns=cols)

    for i in imgname_list:
        df_add = df_orig[df_orig.iloc[0:,2] ==i]
        df_dst = pd.concat([df_dst, df_add])
    return df_dst

df_cont = extract(contlist, df2)
df_exte = extract(extelist, df2)
df_inte = extract(intelist, df2)

In [7]:
import statistics
import math
from scipy import stats
from statsmodels.stats.weightstats import ttest_ind

def statistics_age(age_list):
    print("n=" +str(len(age_list)))
    print("max: "+str(min(age_list)))
    print("min: "+str(max(age_list)))
    print("median: "+str(statistics.median(age_list)))
    print("mean: "+str(statistics.mean(age_list)))
    print("sd: "+str(statistics.stdev(age_list)))
    
print("cont")
cont_age = df_cont.iloc[:,4].tolist()
statistics_age(cont_age)

print("")
print("exte")
exte_age = df_exte.iloc[:,4].tolist()
statistics_age(exte_age)

print("")
print("inte")
inte_age = df_inte.iloc[:,4].tolist()
statistics_age(inte_age)

print("")
print("total")
total_age = cont_age + exte_age + inte_age
statistics_age(total_age)

print("")
print("cont vs exo")
print("p="+str(stats.mannwhitneyu(inte_age, exte_age, use_continuity=True, alternative=None))
)

print("")
print("cont vs eso")
print("p="+str(ttest_ind(cont_age, inte_age)[1]))

cont
n=366
max: 0
min: 83
median: 10.0
mean: 17.959016393442624
sd: 20.37137296036135

exte
n=335
max: 0
min: 82
median: 7
mean: 16.8089552238806
sd: 20.801152456437435

inte
n=294
max: 0
min: 86
median: 5.0
mean: 17.755102040816325
sd: 24.12357034593524

total
n=995
max: 0
min: 86
median: 7
mean: 17.511557788944724
sd: 21.669657621987295

cont vs exo
p=MannwhitneyuResult(statistic=44688.0, pvalue=0.022368943860630472)

cont vs eso
p=0.9063390922312577


In [16]:
stats.shapiro(cont_age)

(0.7602319121360779, 1.0180988986773168e-22)